**Practical 5:
ANOVA (Analysis of Variance)**

** Perform one-way ANOVA to compare means across multiple groups.**

** Conduct post-hoc tests to identify significant differences between group means.**

**ANOVA (F-TEST)**

In [ ]:
import pandas as pd
import scipy.stats as stats

# 1. Read the uploaded dataset
df_anova = pd.read_csv('/content/telco.csv')

# 2. Select the relevant columns for ANOVA (continuous variable and group variable)
df_anova = df_anova[['Total Revenue','Contract']]

# 3. Get the unique groups from the 'Contract' column
grps = pd.unique(df_anova.Contract.values)

# 4. Create a dictionary to separate the 'Total Revenue' data by 'Contract' group
d_data = {grp:df_anova['Total Revenue'][df_anova.Contract == grp] for grp in grps}

# 5. Perform the One-Way ANOVA test
# The groups are: 'Month-to-Month', 'One Year', 'Two Year'
F, p = stats.f_oneway(d_data['Month-to-Month'], d_data['One Year'], d_data['Two Year'])

# 6. Print the results
print("p-value for significance is: ", p)
if p < 0.05:
    print("reject null hypothesis: There is a significant difference in the mean Total Revenue between the contract types.")
else:
    print("accept null hypothesis: There is no significant difference in the mean Total Revenue between the contract types.")

p-value for significance is:  0.0
reject null hypothesis: There is a significant difference in the mean Total Revenue between the contract types.


**Two Way F-test**

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 1. Load the dataset
df_netflix = pd.read_csv('/content/netflix_titles.csv')

# 2. Select relevant columns and clean for missing ratings
df_anova2 = df_netflix[['release_year', 'type', 'rating']].dropna()

# 3. SOLUTION TO THE RANK DEFICIENCY ERROR:
# Identify and remove ratings that appear exclusively in only one 'type' (Movie or TV Show)
crosstab = pd.crosstab(df_anova2['type'], df_anova2['rating'])
exclusive_ratings = crosstab.columns[(crosstab.loc['Movie'] == 0) | (crosstab.loc['TV Show'] == 0)].tolist()
df_anova_cleaned = df_anova2[~df_anova2['rating'].isin(exclusive_ratings)].copy()

# 4. Fit the two-way ANOVA model with the cleaned data
model = ols('release_year ~ C(type) * C(rating)', data=df_anova_cleaned).fit()

# 5. Print the overall model fit statistics
print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

# 6. Calculate and display the Type II ANOVA table
res = sm.stats.anova_lm(model, typ=2)
print("\nANOVA Table (Type II Sum of Squares):")
# CORRECTED LINE: using single quotes to avoid SyntaxError
print(res.to_markdown(numalign='left', stralign='left'))

Overall model F( 17, 7958) =  38.830, p =  0.0000

ANOVA Table (Type II Sum of Squares):
|                   | sum_sq   | df   | F       | PR(>F)      |
|:------------------|:---------|:-----|:--------|:------------|
| C(type)           | 6789.52  | 1    | 105.336 | 1.46595e-24 |
| C(rating)         | 26863.9  | 8    | 52.0975 | 8.06481e-83 |
| C(type):C(rating) | 2176.1   | 8    | 4.22014 | 4.59926e-05 |
| Residual          | 512940   | 7958 | nan     | nan         |
